# ПАМЯТКА
**(1) Генерация тестовых вопросов с помощью Chat GPT 3.5:**
1. Подготовлена тестовая БЗ v.1 - на 1 этапе взята вся инфа из файла "Модель Пиканто". БЗ пердставляет из себя 3 чанка с разделителем <...>, размеры чанков мене  1000 токенов, общий размер БЗ v.1 составляет 2400 токенов.
2. Взят Промпт для генерации тестовых вопросов "Генератор вопросов v.1 (617 t)", общий размер Промпта "Генератор" v.1 составляет 617 токенов.
3. Подаем на вход все 3 чанка с указанным промптом.
4. На выходе получаем 10 тестовых вопросв, сгенерированных моделью 3.5 на основании тестовой БЗ

(за основу взят Colab из 3 урока курса по Дообучению)

Вопросы, Ответы и их Оценка - файл "Тесты 1, 2" (https://docs.google.com/spreadsheets/d/1i7-sDMQYKPaUlgwlKabx-ktKU7u5KI31NXRnXqeEGDM), лист - "Пиканто (05.09 - Галиев Р.)"

# (1) Генерация по БЗ -
**KB_Picanto_test-questions_Eng_v.1**


(из Занятия №3)

Структура базы знаний компании, предназначенной для предоставления ответов на вопросы клиентов в текстовом формате, является ключевым аспектом ее эффективного функционирования. Помните, что для ответов на вопросы будут использоваться чанки, наиболее релевантные запросу пользователя. Токим образом, структура базы знаний должна быть организованной, логической и удобной для поиска информации, чтобы обеспечить максимальную полезность и полноту информации в представленных для анализа чанках.

Вот несколько ключевых принципов, которые следует учесть при формировании структуры базы знаний:

* **Категоризация и классификация:** База знаний должна быть разделена на ясные и логически организованные блоки, соответствующие различным областям знаний или типам вопросов клиентов. Например, это могут быть разделы, связанные с продуктами, услугами, поддержкой, вопросами оплаты и т.д. В каждом блоке могут быть подблоки для дополнительной детализации. Это позволит LangChain легко найти релевантные чанки, соответствующие запросу.

* **Иерархия:** База знаний может быть организована в виде иерархической структуры, где более общие темы располагаются на верхнем уровне, а более специфические вопросы и ответы находятся на более низких уровнях. Так проще контролировать наполненность базы знаний нужной информацией и обновлять ее при необходимости.



## Установка пакетов
Разметка: интеллектуальные чанки

Помимо дробления на чанки "вслепую" при помощи различных сплиттеров, описанных в 1 занятии данного курса, можно дробить тексты на логические, смысловые чанки. Это гораздо более кропотливая и ресурсозатратная работа, но результаты оправдывают ожидания. Её суть заключается в том, что текст изначально разбивается вручную на логические блоки, отделенные определенным разделителем. Затем каждый логический отрезок делится на блоки указанного размера при помощи сплиттера.
Вот, как это можно реализовать:

In [ ]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 chromadb gspread oauth2client nltk pydantic==1.10.8 faiss-cpu==1.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 76.6 MB/s

## Дробление на чанки
Функции (просто запустить все ячейк на исполнение, **ввести АПИ ключ**)

In [ ]:
#@title (import)
import gdown
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken

In [ ]:
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
def split_text(text, max_count, count_type, verbose=0):
    # Функция для подсчета количества слов в фрагменте
    def num_words(fragment):
        return len(fragment.split())

    # Функция для подсчета количества токенов в фрагменте
    def num_tokens(fragment):
        return num_tokens_from_string(fragment, "cl100k_base")

    # Разделение текста на фрагменты, исключая теги HTML
    fragments = [fragment.strip() for fragment in re.split(r"<[^>]+>|[\ufeff]", text) if fragment.strip()]

    # Выбор функции подсчета длины в зависимости от типа подсчета
    length_function = num_words if count_type == "words" else num_tokens

    # Создание объекта разделителя текста
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=0, length_function=length_function)

    # Список для хранения фрагментов текста
    source_chunks = []

    # Обработка каждого фрагмента текста
    for fragment in fragments:
        if verbose:
            # Вывод количества слов/токенов в фрагменте, если включен режим verbose
            count = length_function(fragment)
            print(f"{count_type} in text fragment = {count}\n{'-' * 5}\n{fragment}\n{'=' * 20}")

        # Разбиение фрагмента текста на части заданной длины с помощью разделителя
        # и добавление каждой части в список source_chunks
        #source_chunks.append(Document(page_content=fragment, metadata={}) for chunk in splitter.split_text(fragment))
        source_chunks.extend(Document(page_content=chunk, metadata={}) for chunk in splitter.split_text(fragment))

    # Возвращение списка фрагментов текста
    return source_chunks


def create_embedding(data, max_count, count_type):
    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Возвращает количество токенов в строке"""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    source_chunks = []

    source_chunks = split_text(text=data, max_count=max_count, count_type=count_type, verbose=0)

    # Создание индексов документа
    search_index = FAISS.from_documents(source_chunks, OpenAIEmbeddings(), )

    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    print('\n ===========================================: ')
    print('Количество токенов в документе :', count_token)
    print('ЦЕНА запроса:', 0.0004*(count_token/1000), ' $')
    return search_index

def load_search_indexes(url: str, max_count, count_type) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return create_embedding(text, max_count=max_count, count_type=count_type)


In [ ]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0613":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

def answer_index(system, topic, search_index, temp = 0, verbose = 0, top_similar_documents = 3):

    #Выборка документов по схожести с вопросом
    docs = search_index.similarity_search(topic, k=top_similar_documents)
    if (verbose): print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if (verbose): print('message_content :\n ======================================== \n', message_content)

    messages = [
      {"role": "system", "content": system + f"{message_content}"},
      {"role": "user", "content": topic}
      ]

    # example token count from the function defined above
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f"{num_tokens_from_messages(messages, 'gpt-3.5-turbo-0613')} токенов использовано на вопрос")

    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    temperature=temp
    )
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    if (verbose): print('\n ===========================================: ')
    if (verbose): print('ЦЕНА запроса с ответом :', 0.002*(completion["usage"]["total_tokens"]/1000), ' $')
    if (verbose): print('\n ===========================================: ')
    print('ОТВЕТ : \n', insert_newlines(completion.choices[0].message.content))

    # return completion

In [ ]:
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
max_count = 100            # максимальное кол-во СЛОВ в чанке
count_type = "words"
top_similar_documents = 3

## загрузка доков и Ответ

In [ ]:
# Документ "Тестовая БЗ - KB_Picanto_test-questions_Eng_v.1"
hh_ru_index = load_search_indexes('https://docs.google.com/document/d/1lmISV8QJ9djf9UwbFdBA3IvjtJZc5DYeaNAYWmjxIJc', max_count = max_count, count_type=count_type)


 ===========================================: 
Количество токенов в документе : 2019
ЦЕНА запроса: 0.0008076000000000001  $


In [ ]:
# Промпт для генерации тестовых вопросов

hr_chat_promt = load_document_text('https://docs.google.com/document/d/1ci4ZfqxMJ1K2dbu1Ic5R9HF0W_e0Zeoo')

In [ ]:
ans = answer_index(
    hr_chat_promt,
    'Составь 10 тестовых вопросов так, как если бы спрашивал простой рабочий человек. Вопросы не нумеруй, а между вопросами поставь по 2 разделителя "перевод строки"', # 28 токенов
    hh_ru_index,
    verbose = 1,
    top_similar_documents = top_similar_documents
)


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Kia Picanto
Options
Picanto variants 5 available configurations
Equipment 1 - Classic from 1,334,900 ₽
Engine and transmission
1.0 MPI / 67 hp / Petrol / Manual / Front Wheel Drive
1.0 MPI / 67 hp / Petrol / Automatic / Front Wheel Drive
Main options
Rear seats with 60:40 folding backrests
Steel wheels 14" with decorative hubcaps and tires 175/65 R14
Tire Pressure Monitoring System (TPMS)
Electronic Stability Control (ESC)
Front airbags
Light sensor
Heated windshield washer nozzles
Audio system with 2 speakers
Audio system with 4 speakers
Learn more at https://www.kia.ru/models/picanto/options/
Configuration 2 - Comfort from 1,409,900 ₽
Engine and transmission
1.0 MPI / 67 hp / Petrol / Manual / Front Wheel Drive
1.0 MPI / 67 hp / Petrol / Automatic / Front Wheel Drive
Main options
Steel wheels 14" with decorative hubcaps and tires 175/65 R14
Heated front seats
Heated steering 

## (финиш)
Можно заметить, что чанки разделены точно в соответствии с логическими разделителями, которые мы добавили в текст базы знаний при его разметке. Таким образом, благодаря логической разметке, чанки в Langchain будут подаваться тематическими блоками.

# (2) Прогон по тестам
**Chat GPT отвечает на полученные тестовые вопросы**

БЗ - **KB_Picanto_test-questions_Eng_v.1**

## Установка пакетов

In [ ]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 chromadb gspread oauth2client nltk pydantic==1.10.8 faiss-cpu==1.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 98.6 MB/s

## Дробление на чанки
Функции (просто запустить все ячейк на исполнение, **ввести АПИ ключ**)

In [ ]:
#@title (import)
import gdown
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken

In [ ]:
#@title (ключ API)
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
def split_text(text, max_count, count_type, verbose=0):
    # Функция для подсчета количества слов в фрагменте
    def num_words(fragment):
        return len(fragment.split())

    # Функция для подсчета количества токенов в фрагменте
    def num_tokens(fragment):
        return num_tokens_from_string(fragment, "cl100k_base")

    # Разделение текста на фрагменты, исключая теги HTML
    fragments = [fragment.strip() for fragment in re.split(r"<[^>]+>|[\ufeff]", text) if fragment.strip()]

    # Выбор функции подсчета длины в зависимости от типа подсчета
    length_function = num_words if count_type == "words" else num_tokens

    # Создание объекта разделителя текста
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=0, length_function=length_function)

    # Список для хранения фрагментов текста
    source_chunks = []

    # Обработка каждого фрагмента текста
    for fragment in fragments:
        if verbose:
            # Вывод количества слов/токенов в фрагменте, если включен режим verbose
            count = length_function(fragment)
            print(f"{count_type} in text fragment = {count}\n{'-' * 5}\n{fragment}\n{'=' * 20}")

        # Разбиение фрагмента текста на части заданной длины с помощью разделителя
        # и добавление каждой части в список source_chunks
        #source_chunks.append(Document(page_content=fragment, metadata={}) for chunk in splitter.split_text(fragment))
        source_chunks.extend(Document(page_content=chunk, metadata={}) for chunk in splitter.split_text(fragment))

    # Возвращение списка фрагментов текста
    return source_chunks


def create_embedding(data, max_count, count_type):
    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Возвращает количество токенов в строке"""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    source_chunks = []

    source_chunks = split_text(text=data, max_count=max_count, count_type=count_type, verbose=0)

    # Создание индексов документа
    search_index = FAISS.from_documents(source_chunks, OpenAIEmbeddings(), )

    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    print('\n ===========================================: ')
    print('Количество токенов в документе :', count_token)
    print('ЦЕНА запроса:', 0.0004*(count_token/1000), ' $')
    return search_index

def load_search_indexes(url: str, max_count, count_type) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return create_embedding(text, max_count=max_count, count_type=count_type)


In [ ]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0613":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

def answer_index(system, topic, search_index, temp = 0, verbose = 0, top_similar_documents = 3):

    #Выборка документов по схожести с вопросом
    docs = search_index.similarity_search(topic, k=top_similar_documents)
    if (verbose): print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if (verbose): print('message_content :\n ======================================== \n', message_content)

    messages = [
      {"role": "system", "content": system + f"{message_content}"},
      {"role": "user", "content": topic}
      ]

    # example token count from the function defined above
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f"{num_tokens_from_messages(messages, 'gpt-3.5-turbo-0613')} токенов использовано на вопрос")

    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    temperature=temp
    )
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    if (verbose): print('\n ===========================================: ')
    if (verbose): print('ЦЕНА запроса с ответом :', 0.002*(completion["usage"]["total_tokens"]/1000), ' $')
    if (verbose): print('\n ===========================================: ')
    print('ОТВЕТ : \n', insert_newlines(completion.choices[0].message.content))

    # return completion

In [ ]:
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
max_count = 2400            # максимальное кол-во символов в чанке
count_type = "words"
top_similar_documents = 3

## загрузка доков и Ответ
**Промпт - KB_Picanto_test-questions_Eng_v.1**

In [ ]:
# Документ "Тестовая БЗ - KB_Picanto_test-questions_Eng_v.1"
kia_index = load_search_indexes('https://docs.google.com/document/d/1lmISV8QJ9djf9UwbFdBA3IvjtJZc5DYeaNAYWmjxIJc', max_count = max_count, count_type=count_type)


 ===========================================: 
Количество токенов в документе : 2019
ЦЕНА запроса: 0.0008076000000000001  $


In [ ]:
# Промпт для Ответов модели - "KIA Neural Prompt v.1 (400 т)"

kia_promt = load_document_text('https://docs.google.com/document/d/1tjjJQ0s0qKSdmg2Yd69NXX4_RGWLr0up')

In [ ]:
# Получаем ответы на 10 тестовых вопросов - вопросы пишем "ручками" в соответствующий параметр answer_index()

ans = answer_index(
    kia_promt,
# Пишем текст вопроса вместо ************************ !!!
    'Какие скидки или специальные предложения доступны для покупки Kia Picanto?',
    kia_index,
    verbose = 1,
    top_similar_documents = top_similar_documents
)


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Kia Picanto
Options
Picanto variants 5 available configurations
Equipment 1 - Classic from 1,334,900 ₽
Engine and transmission
1.0 MPI / 67 hp / Petrol / Manual / Front Wheel Drive
1.0 MPI / 67 hp / Petrol / Automatic / Front Wheel Drive
Main options
Rear seats with 60:40 folding backrests
Steel wheels 14" with decorative hubcaps and tires 175/65 R14
Tire Pressure Monitoring System (TPMS)
Electronic Stability Control (ESC)
Front airbags
Light sensor
Heated windshield washer nozzles
Audio system with 2 speakers
Audio system with 4 speakers
Learn more at https://www.kia.ru/models/picanto/options/
Configuration 2 - Comfort from 1,409,900 ₽
Engine and transmission
1.0 MPI / 67 hp / Petrol / Manual / Front Wheel Drive
1.0 MPI / 67 hp / Petrol / Automatic / Front Wheel Drive
Main options
Steel wheels 14" with decorative hubcaps and tires 175/65 R14
Heated front seats
Heated steering 

## (финиш)

# (3) Проверка ответов с помощью Chat GPT
БЗ - **KB_Picanto_test-questions_Eng_v.1**

## Установка пакетов

In [ ]:
!pip  install  tiktoken==0.4.0  langchain==0.0.231 openai==0.27.8 chromadb gspread oauth2client nltk pydantic==1.10.8 faiss-cpu==1.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 95.1 MB/s 

## Дробление на чанки
Функции (просто запустить все ячейк на исполнение, **ввести АПИ ключ**)

In [ ]:
#@title (import)
import gdown
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import getpass
import os
import openai
import tiktoken

In [ ]:
#@title (ключ API)
# Получение ключа API от пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [ ]:
def split_text(text, max_count, count_type, verbose=0):
    # Функция для подсчета количества слов в фрагменте
    def num_words(fragment):
        return len(fragment.split())

    # Функция для подсчета количества токенов в фрагменте
    def num_tokens(fragment):
        return num_tokens_from_string(fragment, "cl100k_base")

    # Разделение текста на фрагменты, исключая теги HTML
    fragments = [fragment.strip() for fragment in re.split(r"<[^>]+>|[\ufeff]", text) if fragment.strip()]

    # Выбор функции подсчета длины в зависимости от типа подсчета
    length_function = num_words if count_type == "words" else num_tokens

    # Создание объекта разделителя текста
    splitter = RecursiveCharacterTextSplitter(chunk_size=max_count, chunk_overlap=0, length_function=length_function)

    # Список для хранения фрагментов текста
    source_chunks = []

    # Обработка каждого фрагмента текста
    for fragment in fragments:
        if verbose:
            # Вывод количества слов/токенов в фрагменте, если включен режим verbose
            count = length_function(fragment)
            print(f"{count_type} in text fragment = {count}\n{'-' * 5}\n{fragment}\n{'=' * 20}")

        # Разбиение фрагмента текста на части заданной длины с помощью разделителя
        # и добавление каждой части в список source_chunks
        #source_chunks.append(Document(page_content=fragment, metadata={}) for chunk in splitter.split_text(fragment))
        source_chunks.extend(Document(page_content=chunk, metadata={}) for chunk in splitter.split_text(fragment))

    # Возвращение списка фрагментов текста
    return source_chunks


def create_embedding(data, max_count, count_type):
    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Возвращает количество токенов в строке"""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    source_chunks = []

    source_chunks = split_text(text=data, max_count=max_count, count_type=count_type, verbose=0)

    # Создание индексов документа
    search_index = FAISS.from_documents(source_chunks, OpenAIEmbeddings(), )

    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    print('\n ===========================================: ')
    print('Количество токенов в документе :', count_token)
    print('ЦЕНА запроса:', 0.0004*(count_token/1000), ' $')
    return search_index

def load_search_indexes(url: str, max_count, count_type) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return create_embedding(text, max_count=max_count, count_type=count_type)


In [ ]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0613":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)

def answer_index(system, topic, search_index, temp = 0, verbose = 0, top_similar_documents = 3):

    #Выборка документов по схожести с вопросом
    docs = search_index.similarity_search(topic, k=top_similar_documents)
    if (verbose): print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if (verbose): print('message_content :\n ======================================== \n', message_content)

    messages = [
      {"role": "system", "content": system + f"{message_content}"},
      {"role": "user", "content": topic}
      ]

    # example token count from the function defined above
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f"{num_tokens_from_messages(messages, 'gpt-3.5-turbo-0613')} токенов использовано на вопрос")

    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    temperature=temp
    )
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    if (verbose): print('\n ===========================================: ')
    if (verbose): print('ЦЕНА запроса с ответом :', 0.002*(completion["usage"]["total_tokens"]/1000), ' $')
    if (verbose): print('\n ===========================================: ')
    print('ОТВЕТ : \n', insert_newlines(completion.choices[0].message.content))

    # return completion

In [ ]:
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
max_count = 2400            # максимальное кол-во символов в чанке
count_type = "words"
top_similar_documents = 3

## загрузка доков и Оценка
**Промпт: "Промпт - Проверка Ответов = Eng (v.1) 511 т"**

In [ ]:
# Документ "Тестовая БЗ - KB_Picanto_test-questions_Eng_v.1"
kia_index = load_search_indexes('https://docs.google.com/document/d/1lmISV8QJ9djf9UwbFdBA3IvjtJZc5DYeaNAYWmjxIJc', max_count = max_count, count_type=count_type)


 ===========================================: 
Количество токенов в документе : 2019
ЦЕНА запроса: 0.0008076000000000001  $


In [ ]:
# Промпт для ПРОВЕРКИ Ответов - "Промпт - Проверка Ответов = Eng (v.1) 511 т"

kia_promt = load_document_text('https://docs.google.com/document/d/1ruT1--vEtCM7B2xVBHuMpBNfGFb2Q3nbU2wiKdFwEic')

In [ ]:
# Получаем ответы на 10 тестовых вопросов - вопросы пишем "ручками" в соответствующий параметр answer_index()

ans = answer_index(
    kia_promt,
# Пишем текст Вопроса и Ответа (из п. (2) ) между 3-х кавычек !!!
    '''
***Вопрос: Какие скидки или специальные предложения доступны для покупки Kia Picanto?***.
**Ответ: На данный момент у нас есть несколько специальных предложений для покупки Kia Picanto. Вот некоторые из них: 1. Финансовая программа "Trade-in". При сдаче вашего старого
 автомобиля в зачет на покупку нового Kia Picanto вы можете получить дополнительную скидку или выгодные условия кредита. 2. Программа лояльности для постоянных клиентов.
 Если вы уже являетесь владельцем автомобиля Kia, вы можете получить специальные условия на покупку нового Kia Picanto. 3. Специальные цены на определенные комплектации
 Kia Picanto. Мы предлагаем сниженные цены на некоторые модели и комплектации автомобиля. 4. Беспроцентный кредит. В рамках этой программы вы можете приобрести Kia
 Picanto в кредит без процентов на определенный период времени. Пожалуйста, обратитесь к вашему ближайшему дилеру Kia для получения более подробной информации о текущих
 специальных предложениях и скидках.**
    ''',
    kia_index,
    verbose = 1,
    top_similar_documents = top_similar_documents
)


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Kia Picanto
Options
Picanto variants 5 available configurations
Equipment 1 - Classic from 1,334,900 ₽
Engine and transmission
1.0 MPI / 67 hp / Petrol / Manual / Front Wheel Drive
1.0 MPI / 67 hp / Petrol / Automatic / Front Wheel Drive
Main options
Rear seats with 60:40 folding backrests
Steel wheels 14" with decorative hubcaps and tires 175/65 R14
Tire Pressure Monitoring System (TPMS)
Electronic Stability Control (ESC)
Front airbags
Light sensor
Heated windshield washer nozzles
Audio system with 2 speakers
Audio system with 4 speakers
Learn more at https://www.kia.ru/models/picanto/options/
Configuration 2 - Comfort from 1,409,900 ₽
Engine and transmission
1.0 MPI / 67 hp / Petrol / Manual / Front Wheel Drive
1.0 MPI / 67 hp / Petrol / Automatic / Front Wheel Drive
Main options
Steel wheels 14" with decorative hubcaps and tires 175/65 R14
Heated front seats
Heated steering 

## (финиш)